## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 23:35:16


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.Pl = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.Php = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 1
model.m = 0.6
model.Ppeak = 10000
#model.Ppeak = (pyo.Param(model.t, initialize =_auxDictionary((data['Inputs']* 1.017).to_numpy()[:,3]))) 
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.PEVdc = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Ppeakred = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Ppeakred2 = pyo.Var(domain = pyo.Reals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _power_discharging_limit2(m,ev,t): 
    return m.PEVdc[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_discharging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_discharging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
    #return m.EEVmin[ev] <= m.EEV[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)

def _energy_EVs_trips(m,t): 
    return m.Ppeakred2 >= m.Pl[t] + sum(m.PEV[ev,t] - m.PEVdc[ev,t] for ev in np.arange(1, n_evs + 1))
model.energy_EVs_trips = pyo.Constraint(model.t, rule = _energy_EVs_trips)

## Objective function

In [11]:
def _FOag(m):
    return m.Ppeakred2 + sum([m.PEVdc[ev, t] for t in np.arange(1, n_time + 1) for ev in np.arange(1, n_evs + 1)])*0.1 + sum([m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2
                                                                                             for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmp7k6ycfs2
  Lower bound: 602743856.5657594
  Upper bound: 602743856.5657594
  Number of objectives: 1
  Number of constraints: 507025
  Number of variables: 432002
  Number of nonzeros: 1083025
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 4.14
  Termination condition: optimal
  Termination message: Dual simplex - Optimal\x3a Objective = 6.0274385657e+08
  Error rc: 0
  Time: 6.649292945861816
# ----------------------------------------------------------
#   Solution Information
# --------------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

602743856.565759

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 23:37:04
Dif: 0:01:48


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
PEVdc_df = ext_pyomo_vals(model.PEVdc)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

discharge_cost = sum([PEVdc_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                      for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))
print('Discharge cost: {}'.format(discharge_cost))

Charge cost: 3618294.501650523
Discharge cost: 0.0


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy())))

Total Charge: 21080336.064517155


In [18]:
print("Total Discharge: {}".format(np.sum(PEVdc_df.to_numpy())))

Total Discharge: 0.0


In [19]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,42386.148255,42386.148255,42386.148255,42386.148255,42386.148255,42386.148255,42386.148255,45986.148255,49241.555235,50227.703490,...,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,45000.0
2,9960.000000,11263.752480,11263.752480,11263.752480,11263.752480,11263.752480,11263.752480,11263.752480,11263.752480,11263.752480,...,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,9960.0
3,40200.000000,47400.000000,47400.000000,47400.000000,47400.000000,47400.000000,47400.000000,50835.300800,50835.300800,47290.200533,...,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,40200.0
4,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,...,47147.260680,47147.260680,47147.260680,47147.260680,47147.260680,47147.260680,47147.260680,47147.260680,47147.260680,45000.0
5,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46720.845840,46720.845840,46720.845840,46720.845840,46720.845840,...,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,...,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,47776.206600,47776.206600,45000.0
2997,46368.994700,46368.994700,46368.994700,46368.994700,46368.994700,46368.994700,46368.994700,46368.994700,46368.994700,46368.994700,...,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.0
2998,42000.000000,42000.000000,42000.000000,49200.000000,53674.929120,60874.929120,60874.929120,60874.929120,60874.929120,60874.929120,...,55482.092229,52785.673783,52785.673783,50089.255337,50089.255337,50089.255337,47392.836891,44696.418446,42000.000000,42000.0
2999,46132.594020,46132.594020,46132.594020,46132.594020,46132.594020,46132.594020,46132.594020,46132.594020,46132.594020,46132.594020,...,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.0


In [20]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.0,3600.0000,3255.40698,3600.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
2,0.00000,1303.75248,0.0,0.0,0.00000,0.00000,0.0,0.0000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
3,0.00000,7200.00000,0.0,0.0,0.00000,0.00000,0.0,3435.3008,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
4,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.0000,0.00000,0.0,...,4294.52136,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
5,0.00000,0.00000,0.0,0.0,0.00000,520.84584,0.0,0.0000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.0000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,5552.4132,0.0,0.0
2997,2737.98940,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.0000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
2998,0.00000,0.00000,0.0,7200.0,4474.92912,7200.00000,0.0,0.0000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
2999,2265.18804,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.0000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0


In [21]:
PEVdc_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
PEV_df.sum()

1     7.944905e+05
2     1.177205e+06
3     1.023421e+06
4     1.055220e+06
5     9.097466e+05
6     9.631617e+05
7     9.867536e+05
8     9.631335e+05
9     1.013283e+06
10    9.580696e+05
11    8.295262e+05
12    5.674973e+05
13    9.088421e+05
14    9.053591e+05
15    8.092909e+05
16    7.511425e+05
17    6.993573e+05
18    8.058515e+05
19    7.995854e+05
20    8.802082e+05
21    9.987189e+05
22    1.057491e+06
23    6.699716e+05
24    5.530085e+05
dtype: float64

In [23]:
PEVdc_df.sum()

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

In [24]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [25]:
Etriprelax_df.sum()

1          0.000000
2          0.000000
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11         0.000000
12         0.000000
13      1504.726000
14     10688.567539
15     38076.567566
16     55382.034645
17     25474.659849
18     63663.106584
19     74379.635074
20     53301.261327
21     54985.810789
22     30085.376636
23      2848.957287
24    185046.730320
dtype: float64

In [26]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [27]:
Eminsocrelax_df.sum()

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

In [28]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [29]:
Etripn_df.sum()

1     1.673788e+06
2     2.896256e+04
3     0.000000e+00
4     0.000000e+00
5     0.000000e+00
6     0.000000e+00
7     0.000000e+00
8     0.000000e+00
9     0.000000e+00
10    5.465601e+05
11    1.315866e+06
12    0.000000e+00
13    8.313551e+05
14    3.272904e+06
15    1.418766e+06
16    1.735838e+06
17    5.656545e+05
18    1.100017e+06
19    3.648812e+05
20    1.147978e+06
21    1.388378e+06
22    1.200184e+06
23    1.404881e+06
24    3.603640e+06
dtype: float64

In [30]:
Ppeakred_df = ext_pyomo_vals(model.Ppeakred)

In [31]:
Ppeakred_df

,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
10,0


In [32]:
Ppeakred2_df = ext_pyomo_vals(model.Ppeakred2)

In [33]:
Ppeakred2_df

,0
None,7.306423e+06


## Save results in csv files

In [34]:
import os 
folder = 'results_Peak_tot_V2G_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEVdc_df.to_csv(folder + '/PEVdc.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')
PEVdc_df.sum().to_csv(folder + '/PEVdc_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')

Ppeakred_df.to_csv(folder + '/Ppeakread.csv')

Pcons_df = PEV_df.sum() + data['Inputs'].to_numpy()[:,2] - PEVdc_df.sum()
Pcons_df.to_csv(folder + '/Pcons.csv')